In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style()

In [4]:
table = pd.read_csv("titanic_features_table.csv")
table

,年龄,兄弟姐妹/配偶的个数,父母/小孩个数,票价,1等舱,2等舱,3等舱,登船港口为空,登船为S港,登船为Q港,登船为nan港,女性,男性,index,是否生还
0,22.0,1,0,7.2500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0,0
1,38.0,1,0,71.2833,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1,1
2,26.0,0,0,7.9250,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,2,1
3,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3,1
4,35.0,0,0,8.0500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,0.0,0,0,8.0500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,413,0
1305,39.0,0,0,108.9000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,414,1
1306,38.5,0,0,7.2500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,415,0
1307,0.0,0,0,8.0500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,416,0


In [8]:
table.columns

Index(['年龄', '兄弟姐妹/配偶的个数', '父母/小孩个数', '票价', '1等舱', '2等舱', '3等舱', '登船港口为空',
       '登船为S港', '登船为Q港', '登船为nan港', '女性', '男性', 'index', '是否生还'],
      dtype='object')

In [14]:
X = np.matrix(table[['年龄', '兄弟姐妹/配偶的个数', '父母/小孩个数', '票价', '1等舱', '2等舱', '3等舱', '登船港口为空',
       '登船为S港', '登船为Q港', '登船为nan港', '女性', '男性', 'index']])
Y = np.matrix(table[['是否生还']])

In [17]:
from sklearn.model_selection import train_test_split
train_X,test_x,train_Y,test_y = train_test_split(X,Y,test_size=0.3193,random_state=42)

In [18]:
import xgboost as xgb


gamma = 0
### 自动计算alpha值的取值范围 取负例的比例
alpha = (train_Y==0).sum()/train_Y.size

def logistic_obj(p, dtrain):
    y = dtrain.get_label()
    p = 1.0 / (1.0 + np.exp(-p))
    grad = p * (1 - p) * (alpha * gamma * y * (1 - p) ** gamma * np.log(p) / (1 - p) - alpha * y * (
                1 - p) ** gamma / p - gamma * p ** gamma * (1 - alpha) * (1 - y) * np.log(1 - p) / p + p ** gamma * (
                                      1 - alpha) * (1 - y) / (1 - p))
    hess = p * (1 - p) * (p * (1 - p) * (
                -alpha * gamma ** 2 * y * (1 - p) ** gamma * np.log(p) / (1 - p) ** 2 + alpha * gamma * y * (
                    1 - p) ** gamma * np.log(p) / (1 - p) ** 2 + 2 * alpha * gamma * y * (1 - p) ** gamma / (
                            p * (1 - p)) + alpha * y * (1 - p) ** gamma / p ** 2 - gamma ** 2 * p ** gamma * (
                            1 - alpha) * (1 - y) * np.log(1 - p) / p ** 2 + 2 * gamma * p ** gamma * (1 - alpha) * (
                            1 - y) / (p * (1 - p)) + gamma * p ** gamma * (1 - alpha) * (1 - y) * np.log(
            1 - p) / p ** 2 + p ** gamma * (1 - alpha) * (1 - y) / (1 - p) ** 2) - p * (
                                      alpha * gamma * y * (1 - p) ** gamma * np.log(p) / (1 - p) - alpha * y * (
                                          1 - p) ** gamma / p - gamma * p ** gamma * (1 - alpha) * (1 - y) * np.log(
                                  1 - p) / p + p ** gamma * (1 - alpha) * (1 - y) / (1 - p)) + (1 - p) * (
                                      alpha * gamma * y * (1 - p) ** gamma * np.log(p) / (1 - p) - alpha * y * (
                                          1 - p) ** gamma / p - gamma * p ** gamma * (1 - alpha) * (1 - y) * np.log(
                                  1 - p) / p + p ** gamma * (1 - alpha) * (1 - y) / (1 - p)))
    return grad, hess

params = { "learning_rate":0.1
               , "n_estimators":11 #即基评估器的数量。这个参数对随机森林模型的精确性影响是单调的，n_estimators越 大，模型的效果往往越好。但是相应的，任何模型都有决策边  n_estimators达到一定的程度之后，随机森林的 精确性往往不在上升或开始波动，并且，n_estimators越大，需要的计算量和内存也越大，训练的时间也会越来越 长。对于这个参数，我们是渴望在训练难度和模型效果之间取得平衡。
               , "max_depth":12 #构建树的深度，越大越容易过拟合
               , "min_child_weight":0.8 #孩子节点中最小的样本权重和。如果一个叶子节点的样本权重和小于min_child_weight则拆分过程结束
#                , "num_class ":1#类别数，与 multisoftmax 并用
#                , "gamma":0 #损失下降多少才进行分裂， 控制叶子节点的个数
               , "subsample":0.8 #随机采样训练样本
               , "colsample_bytree":1 #生成树时进行的列采样
#                , "objective":'binary:logistic' # {'binary:logistic'}是二分类的问题，{'multi:softmax',}是多分类的问题 这个是优化目标，必须得有，因为xgboost里面有求一阶导数和二阶导数，其实就是这个。
               , "nthread":4 #cpu 线程数
               , "scale_pos_weight":1.0 #负样本总数/正样本总数 。若训练负样本总数是500 ，正样本总数100，那么设置 scale_pos_weigh为 5
               , "lambda_weight":2 # 正则化参数
               , "eta":0.001 # 如同学习率
#               
#                , metrics='auc'
               , "eval_metric":"auc"
#                , "silent ": 0 # ,设置成1则没有运行信息输出，最好是设置为0.
               , "seed":27
        }
dtrain = xgb.DMatrix(train_X,label=train_Y)
dtest = xgb.DMatrix(test_x,label=test_y)
clf = xgb.train(params=params,dtrain=dtrain,num_boost_round=100,evals=[(dtrain,"train")],verbose_eval=True,obj=logistic_obj)

[14:24:18] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learner.cc:627: 
Parameters: { "lambda_weight", "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.88311
[1]	train-auc:0.91106
[2]	train-auc:0.91165
[3]	train-auc:0.92380
[4]	train-auc:0.91959
[5]	train-auc:0.91952
[6]	train-auc:0.92433
[7]	train-auc:0.93232
[8]	train-auc:0.93511
[9]	train-auc:0.93493
[10]	train-auc:0.93785
[11]	train-auc:0.94545
[12]	train-auc:0.94936
[13]	train-auc:0.95177
[14]	train-auc:0.95294
[15]	train-auc:0.95761
[16]	train-auc:0.95930
[17]	train-auc:0.96079
[18]	train-auc:0.96193
[19]	train-auc:0.96344
[20]	train-auc:0.96615
[21]	train-auc:0.96714
[22]	train-auc:0.96833
[23]	train-a

* ## 2 AUC 指标
### 手动计算auc得分
#### 首先要明白ＡＵＣ的物理含义不仅是ＲＯＣ曲线下的面积，ＡＵＣ还有另外一个物理含义就是：给定正样本Ｍ个，负样本Ｎ个，以及他们的预测概率（０－１）之间，那么ＡＵＣ的含义就是所有穷举所有的正负样本对，如果正样本的预测概率大于负样本的预测概率，那么就＋１；如果如果正样本的预测概率等于负样本的预测概率，那么就＋0.5,　如果正样本的预测概率小于负样本的预测概率，那么就＋０；最后把统计处理的个数除以Ｍ×Ｎ就得到我们的ＡＵＣ，公式描述如下：
## $ \frac{\sum{I(P_{正样本},P_{负样本})}}{m*n}$

In [22]:
def AUC(label, pre):
    pos = [i for i in range(len(label)) if label[i] == 1]
    neg = [i for i in range(len(label)) if label[i] == 0]
    #计算正样本和负样本的索引，以便索引出之后的概率值
    auc = 0
    for i in pos:
        for j in neg:
            if pre[i] > pre[j]:
                auc += 1
            elif pre[i] == pre[j]:
                auc += 0.5
    return auc / (len(pos)*len(neg))

In [52]:
scores = []
predict_proba = clf.predict(dtest)
for prob in np.arange(0.1,1,0.01):
    predict_y = (predict_proba>prob).astype(int)
    accuracy = (test_y==pred_y).sum()/predict_proba.size
    scores.append(accuracy)
    probs.append(prob)

max_prob = probs[scores.index(max(scores))]
predict_y = np.array([pr>max_prob and 1 or 0 for pr in predict_y])
predict_y.size

418

In [45]:
AUC(test_y,predict_proba)

0.9274422670649086

### scikit-learn 计算auc

In [46]:
from sklearn.metrics import roc_curve
[fpr,tpr,thr]=roc_curve(test_y,predict_proba)
from sklearn.metrics import auc
f"auc得分：{auc(fpr,tpr)}"

'auc得分：0.9274422670649085'

* ## 3 准确率计算

In [47]:
from sklearn.metrics import accuracy_score
f"accuracy得分返回正确分类的比率：{accuracy_score(test_y,predict_y)}"

'accuracy得分返回正确分类的比率：0.8708133971291866'

#### 3.2 手动计算准确率 
#### $ \frac{\sum_{n}{Y_{i}=P_{i}}}{n} $ 

In [68]:
def accuracy(lable,predict):
    m = len(predict)
    return (np.ravel(lable)==np.ravel(predict)).sum()/m

accuracy(test_y,predict_y)

0.8708133971291866